In [ ]:
from py123d.datatypes.scene.arrow.arrow_scene_builder import ArrowSceneBuilder
from py123d.datatypes.scene.scene_filter import SceneFilter


from py123d.common.multithreading.worker_sequential import Sequential
# from py123d.datatypes.sensors.camera.pinhole_camera import PinholeCameraType 

In [ ]:

# splits = ["wopd_val"]
# splits = ["carla_test"]
# splits = ["nuscenes-mini_val", "nuscenes-mini_train"]
# splits = ["av2-sensor-mini_train"]
# splits = ["pandaset_train"]

# log_names = None

from py123d.common.multithreading.worker_ray import RayDistributed


splits = ["kitti360"]

log_names = ["2013_05_28_drive_0000_sync"]
scene_uuids = None

scene_filter = SceneFilter(
    split_names=splits,
    log_names=log_names,
    scene_uuids=scene_uuids,
    duration_s=10.0,
    history_s=0.0,
    timestamp_threshold_s=30,
    shuffle=True,
    # camera_types=[CameraType.CAM_F0],
)
scene_builder = ArrowSceneBuilder()
worker = Sequential()
# worker = RayDistributed()
scenes = scene_builder.get_scenes(scene_filter, worker)

print(f"Found {len(scenes)} scenes")

In [ ]:
from typing import List, Optional, Tuple

import matplotlib.pyplot as plt
import numpy as np

from py123d.geometry import Point2D
from py123d.visualization.color.color import BLACK, DARK_GREY, DARKER_GREY, LIGHT_GREY, NEW_TAB_10, TAB_10
from py123d.visualization.color.config import PlotConfig
from py123d.visualization.color.default import CENTERLINE_CONFIG, MAP_SURFACE_CONFIG, ROUTE_CONFIG
from py123d.visualization.matplotlib.observation import (
    add_box_detections_to_ax,
    add_default_map_on_ax,
    add_ego_vehicle_to_ax,
    add_traffic_lights_to_ax,
)
from py123d.visualization.matplotlib.utils import add_shapely_linestring_to_ax, add_shapely_polygon_to_ax
from py123d.datatypes.maps.abstract_map import AbstractMap
from py123d.datatypes.maps.abstract_map_objects import AbstractLane, AbstractLaneGroup
from py123d.datatypes.maps.gpkg.gpkg_map_objects import GPKGIntersection
from py123d.datatypes.maps.map_datatypes import MapLayer
from py123d.datatypes.scene.abstract_scene import AbstractScene


import shapely.geometry as geom

LEFT_CONFIG: PlotConfig = PlotConfig(
    fill_color=TAB_10[2],
    fill_color_alpha=1.0,
    line_color=TAB_10[2],
    line_color_alpha=0.5,
    line_width=1.0,
    line_style="-",
    zorder=3,
)

RIGHT_CONFIG: PlotConfig = PlotConfig(
    fill_color=TAB_10[3],
    fill_color_alpha=1.0,
    line_color=TAB_10[3],
    line_color_alpha=0.5,
    line_width=1.0,
    line_style="-",
    zorder=22,
)


LANE_CONFIG: PlotConfig = PlotConfig(
    fill_color=BLACK,
    fill_color_alpha=1.0,
    line_color=BLACK,
    line_color_alpha=0.0,
    line_width=0.0,
    line_style="-",
    zorder=5,
)

ROAD_EDGE_CONFIG: PlotConfig = PlotConfig(
    fill_color=DARKER_GREY,
    fill_color_alpha=1.0,
    line_color=DARKER_GREY,
    line_color_alpha=1.0,
    line_width=1.0,
    line_style="-",
    zorder=3,
)

ROAD_LINE_CONFIG: PlotConfig = PlotConfig(
    fill_color=NEW_TAB_10[5],
    fill_color_alpha=1.0,
    line_color=NEW_TAB_10[5],
    line_color_alpha=1.0,
    line_width=1.5,
    line_style="-",
    zorder=3,
)


def add_debug_map_on_ax(
    ax: plt.Axes,
    map_api: AbstractMap,
    point_2d: Point2D,
    radius: float,
    route_lane_group_ids: Optional[List[int]] = None,
) -> None:
    layers: List[MapLayer] = [
        # MapLayer.LANE,
        MapLayer.LANE_GROUP,
        MapLayer.GENERIC_DRIVABLE,
        MapLayer.CARPARK,
        # MapLayer.CROSSWALK,
        # MapLayer.INTERSECTION,
        MapLayer.WALKWAY,
        MapLayer.ROAD_EDGE,
        MapLayer.ROAD_LINE,
    ]
    x_min, x_max = point_2d.x - radius, point_2d.x + radius
    y_min, y_max = point_2d.y - radius, point_2d.y + radius
    patch = geom.box(x_min, y_min, x_max, y_max)
    map_objects_dict = map_api.query(geometry=patch, layers=layers, predicate="intersects")
    # print(map_objects_dict[MapLayer.ROAD_EDGE])

    for layer, map_objects in map_objects_dict.items():
        for map_object in map_objects:
            try:
                if layer in [
                    MapLayer.GENERIC_DRIVABLE,
                    MapLayer.CARPARK,
                    MapLayer.CROSSWALK,
                    # MapLayer.INTERSECTION,
                    MapLayer.WALKWAY,
                ]:
                    add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])

                if layer in [MapLayer.LANE_GROUP]:
                    map_object: AbstractLaneGroup
                    add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])

                    if map_object.intersection is not None:
                        add_shapely_polygon_to_ax(ax, map_object.intersection.shapely_polygon, ROUTE_CONFIG)

                        for lane in map_object.lanes:
                            add_shapely_linestring_to_ax(ax, lane.centerline.linestring, CENTERLINE_CONFIG)

                # if layer in [MapLayer.LANE]:
                #     add_shapely_linestring_to_ax(ax, map_object.centerline.linestring, CENTERLINE_CONFIG)
                #     add_shapely_polygon_to_ax(ax, map_object.shapely_polygon, MAP_SURFACE_CONFIG[layer])

                if layer in [MapLayer.ROAD_EDGE]:
                    add_shapely_linestring_to_ax(ax, map_object.polyline_3d.linestring, ROAD_EDGE_CONFIG)

                if layer in [MapLayer.ROAD_LINE]:
                    # line_type = int(map_object.road_line_type)
                    # plt_config = PlotConfig(
                    #     fill_color=NEW_TAB_10[line_type % (len(NEW_TAB_10) - 1)],
                    #     fill_color_alpha=1.0,
                    #     line_color=NEW_TAB_10[line_type % (len(NEW_TAB_10) - 1)],
                    #     line_color_alpha=1.0,
                    #     line_width=1.5,
                    #     line_style="-",
                    #     zorder=10,
                    # )
                    add_shapely_linestring_to_ax(ax, map_object.polyline_3d.linestring, ROAD_LINE_CONFIG)

            except Exception:
                import traceback

                print(f"Error adding map object of type {layer.name} and id {map_object.object_id}")
                traceback.print_exc()

    # ax.set_title(f"Map: {map_api.map_name}")


def _plot_scene_on_ax(ax: plt.Axes, scene: AbstractScene, iteration: int = 0, radius: float = 80) -> plt.Axes:

    ego_vehicle_state = scene.get_ego_state_at_iteration(iteration)
    box_detections = scene.get_box_detections_at_iteration(iteration)
    map_api = scene.get_map_api()

    point_2d = ego_vehicle_state.bounding_box.center.state_se2.point_2d
    if map_api is not None:
        add_debug_map_on_ax(ax, scene.get_map_api(), point_2d, radius=radius, route_lane_group_ids=None)


        # add_default_map_on_ax(ax, map_api, point_2d, radius=radius, route_lane_group_ids=None)
    # add_traffic_lights_to_ax(ax, traffic_light_detections, scene.get_map_api())

    add_box_detections_to_ax(ax, box_detections)
    add_ego_vehicle_to_ax(ax, ego_vehicle_state)

    zoom = 1.0
    ax.set_xlim(point_2d.x - radius * zoom, point_2d.x + radius * zoom)
    ax.set_ylim(point_2d.y - radius * zoom, point_2d.y + radius * zoom)

    ax.set_aspect("equal", adjustable="box")
    return ax


def plot_scene_at_iteration(
    scene: AbstractScene, iteration: int = 0, radius: float = 80
) -> Tuple[plt.Figure, plt.Axes]:

    size = 10

    fig, ax = plt.subplots(figsize=(size, size))
    _plot_scene_on_ax(ax, scene, iteration, radius)
    return fig, ax


# scene_index =
iteration = 1

scale = 10
fig, ax = plt.subplots(1, 1, figsize=(scale, scale))
scene = np.random.choice(scenes)
_plot_scene_on_ax(ax, scene, iteration, radius=80)
# _plot_scene_on_ax(ax[1], scene, iteration, radius=50)
# _plot_scene_on_ax(ax[2], scene, iteration,
#  radius=100)

plt.show()

In [ ]:

map_api = scene.get_map_api()
map_api: AbstractMap

intersection = map_api.get_map_object("562", MapLayer.INTERSECTION)


lane_groups = intersection.lane_groups



In [ ]:
import shapely
from py123d.conversion.utils.map_utils.road_edge.road_edge_2d_utils import get_road_edge_linear_rings

# from py123d.conversion.utils.map_utils.road_edge.road_edge_3d_utils import lift_road_edges_to_3d
from py123d.conversion.utils.map_utils.road_edge.road_edge_3d_utils import (
    _interpolate_z_on_segment,
    _split_continuous_segments,
)
from py123d.geometry.geometry_index import Point3DIndex
from py123d.geometry.occupancy_map import OccupancyMap2D
from py123d.geometry.polyline import Polyline3D


fix, ax = plt.subplots()


def lift_outlines_to_3d(
    outlines_2d: List[shapely.LinearRing],
    boundaries: List[Polyline3D],
    max_distance: float = 10.0,
) -> List[Polyline3D]:
    """Lift 2D road edges to 3D by querying elevation from boundary segments.

    :param road_edges_2d: List of 2D road edge geometries.
    :param boundaries: List of 3D boundary geometries.
    :param max_distance: Maximum 2D distance for edge-boundary association.
    :return: List of lifted 3D road edge geometries.
    """

    outlines_3d: List[Polyline3D] = []

    if len(outlines_2d) >= 1 and len(boundaries) >= 1:

        # 1. Build comprehensive spatial index with all boundary segments
        # NOTE @DanielDauner: We split each boundary polyline into small segments.
        # The spatial indexing uses axis-aligned bounding boxes, where small geometries lead to better performance.
        boundary_segments = []
        for boundary in boundaries:
            coords = boundary.array.reshape(-1, 1, 3)
            segment_coords_boundary = np.concatenate([coords[:-1], coords[1:]], axis=1)
            boundary_segments.append(segment_coords_boundary)

        boundary_segments = np.concatenate(boundary_segments, axis=0)
        boundary_segment_linestrings = shapely.creation.linestrings(boundary_segments)
        occupancy_map = OccupancyMap2D(boundary_segment_linestrings)

        for linear_ring in outlines_2d:
            points_2d = np.array(linear_ring.coords, dtype=np.float64)
            points_3d = np.zeros((len(points_2d), len(Point3DIndex)), dtype=np.float64)
            points_3d[..., Point3DIndex.XY] = points_2d

            # 3. Batch query for all points
            query_points = shapely.creation.points(points_2d)
            results = occupancy_map.query_nearest(query_points, max_distance=max_distance, exclusive=True)

            for query_idx, geometry_idx in zip(*results):
                query_point = query_points[query_idx]
                segment_coords = boundary_segments[geometry_idx]
                best_z = _interpolate_z_on_segment(query_point, segment_coords)
                points_3d[query_idx, 2] = best_z

            outlines_3d.append(Polyline3D.from_array(points_3d))

    return outlines_3d


def _extract_intersection_outline(lane_groups: List[AbstractLaneGroup], junction_id: str = 0) -> Polyline3D:
    """Helper method to extract intersection outline in 3D from lane group helpers."""

    # 1. Extract the intersection outlines in 2D
    intersection_polygons: List[shapely.Polygon] = [
        lane_group_helper.shapely_polygon for lane_group_helper in lane_groups
    ]
    # for intersection_polygon in intersection_polygons:
    #     ax.plot(*intersection_polygon.exterior.xy)

    # for lane_group_helper in lane_groups:
    #     ax.plot(*lane_group_helper.outline.linestring.xy, color="blue")
    intersection_edges = get_road_edge_linear_rings(intersection_polygons, add_interiors=False)

    # for linear_ring in intersection_edges:
    #     ax.plot(*linear_ring.xy, color="blue")

    # 2. Lift the 2D outlines to 3D
    lane_group_outlines: List[Polyline3D] = [lane_group_helper.outline_3d for lane_group_helper in lane_groups]
    intersection_outlines = lift_outlines_to_3d(intersection_edges, lane_group_outlines)

    print(len(intersection_outlines))

    # NOTE: When the intersection has multiple non-overlapping outlines, we cannot return a single outline in 3D.
    # For now, we return the longest outline.

    valid_outlines = [outline for outline in intersection_outlines if outline.array.shape[0] > 2]
    assert len(valid_outlines) > 0, f"No valid intersection outlines found for Junction {junction_id}!"

    longest_outline = max(valid_outlines, key=lambda outline: outline.length)

    # for linear_ring in intersection_outlines:
    #     ax.plot(*linear_ring.linestring.xy, color="red")

    # ax.plot(*longest_outline.linestring.xy, color="red")
    # longest_outline.line
    print(longest_outline.array[:, 2])
    return longest_outline


_extract_intersection_outline(lane_groups)